In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

지금까지 간단한 ML과 DL 문제만을 풀다가 처음으로 노트북을 올리게되었습니다.<br>
지금껏 다른 노트북등을 통해 정리한 기법을 토대로 이번 노트북이 작성되었습니다.

# 1. Import modules

In [ ]:
import sklearn
import keras
import warnings
import missingno as msno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter

plt.style.use('seaborn')
sns.set(font_scale=1.5)

# ignore warnings
warnings.filterwarnings('ignore')

%matplotlib inline

os.listdir("../input")

In [ ]:
df_train = pd.read_csv(
    "../input/tabular-playground-series-jan-2021/train.csv")
df_test = pd.read_csv(
    "../input/tabular-playground-series-jan-2021/test.csv")
df_submit = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

# 2. Data check

1. Data check

In [ ]:
df_train.head()
# 잘 불러졌는지 확인합니다.

In [ ]:
df_test.head()

In [ ]:
df_submit.head()

In [ ]:
df_train.shape, df_test.shape, df_submit.shape
# train 데이터는 300000개의 데이터와 16개의 feature
# test 데이터는 200000개의 데이터와 15개의 feature가 있습니다.

In [ ]:
df_train.columns

In [ ]:
df_train.describe()

2. Detect missing value

In [ ]:
df_train.info()
# 결측치는 없습니다.

3. Detect outliers

In [ ]:
def detect_outliers(df, n, features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1

        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[col] < Q1 - outlier_step)
                              | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)

    return multiple_outliers

In [ ]:
Outliers_to_drop = detect_outliers(df_train, 2, ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14', 'target'])

In [ ]:
# train 데이터의 이상치를 탐색합니다.
# IQR(튜키의 방법)을 이용한 함수를 지정하여 이상치 탐색을 수행합니다.

df_train.loc[Outliers_to_drop].sum()
# 학습데이터에 이상치가 없습니다.

4. Skewness

In [ ]:
for col in df_train.columns:
    print('{:15}'.format(col),
          'Skewness: {:05.2f}'.format(df_train[col].skew()),
          '   ',
          'Kurtosis: {:06.2f}'.format(df_train[col].kurt())
          )

# Skewness(왜도), Kurtosis(첨도)를 확인합니다.
# 이는 분포가 얼마나 비대칭을 띄는가 알려주는 척도입니다. (왜도: a=0이면 정규분포, a<0 이면 오른쪽으로 치우침, a>0이면 왼쪽으로 치우침)
# 데이터의 왜도는 크지 않습니다.

# 3. EDA

1. Correlation Heat Map

In [ ]:
corr_data = df_train[['target', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']]

colormap = plt.cm.PuBu
sns.set(font_scale=1.0)

k = 15
cols = corr_data.corr().nlargest(k, 'target')['target'].index
print(cols)
cm = np.corrcoef(df_train[cols].values.T)
f, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(cm, vmax=.8, linewidths=0.1, square=True, annot=True, cmap=colormap,
            linecolor="white", xticklabels=cols.values, annot_kws={'size': 10}, yticklabels=cols.values)

# Heat Map은 seaborn 덕분에 직관적으로 이해가 가능하여 변수 간 상관관계에 대하여 쉽게 알 수 있습니다.
# 현재 target은 대부분의 feature에 대해 명확한 상관관계를 가지고 있지 않습니다.

# 4. Modeling

In [ ]:
from sklearn.model_selection import train_test_split

X_train = df_train.drop(["id", "target"], axis=1).values
target_label = df_train["target"].values
X_test = df_test.drop("id", axis=1).values

X_train.shape, X_test.shape

In [ ]:
X_tr, X_vld, y_tr, y_vld = train_test_split(
    X_train, target_label, test_size=0.2, random_state=2000)

# Test하기 전 Validation 과정을 겨처줍니다.
# train데이터의 20%를 validation으로 주고 80%을 train으로 남겨주어 분리해줍니다.

y_tr.shape, y_vld.shape

1. xgboost

In [ ]:
from sklearn.model_selection import cross_val_score
import xgboost
from sklearn import metrics

regressor = xgboost.XGBRegressor()
regressor.fit(X_tr, y_tr)

# XGBoost 모델을 만들어줍니다.

In [ ]:
y_hat = regressor.predict(X_tr)

plt.scatter(y_tr, y_hat, alpha=0.2)
plt.xlabel('Targets (y_tr)', size=18)
plt.ylabel('Predictions (y_hat)', size=18)
plt.show()

# 예측 된 y 값 (y_hat)에 대한 Scatter Plot을 그려봅니다.

In [ ]:
regressor.score(X_tr, y_tr)

In [ ]:

y_hat_test = regressor.predict(X_vld)


plt.scatter(y_vld, y_hat_test, alpha=0.2)
plt.xlabel('Targets (y_vld)', size=18)
plt.ylabel('Predictions (y_hat_test)', size=18)
plt.show()

# validation으로 예측해봅니다.

In [ ]:
regressor.score(X_vld, y_vld)

accuracies = cross_val_score(estimator=regressor, X=X_tr, y=y_tr, cv=10)

# k-fold validation을 수행합니다.

In [ ]:
print(accuracies.mean())
print(accuracies.std())

# 정확도를 확인해봅니다.

# 5. Submission

In [ ]:
df_submit.head()

In [ ]:
prediction = regressor.predict(X_test)
df_submit['target'] = prediction

In [ ]:
df_submit.to_csv('my_first_submission.csv', index=False)